# 课程练习

# 1. 路径规划

In [1]:
BEIJING, CHANGCHUN, MULUMUQI, WUHAN, GUNAGHZOU, SHENZHEN, BANGKOK, SHANGHAI, NEWYORK = """
BEIJING CHANGCHUN MULUMUQI WUHAN GUANGZHOU SHENZHEN BANGKOK SHANGHAI NEWYORK
""".split()

In [2]:
dictionary = {}

In [3]:
connection={
    CHANGCHUN:[BEIJING],
    BEIJING:[MULUMUQI, CHANGCHUN, WUHAN, SHENZHEN, NEWYORK],
    MULUMUQI:[BEIJING],
    NEWYORK: [BEIJING, SHANGHAI],
    SHANGHAI: [NEWYORK, WUHAN],
    WUHAN: [SHANGHAI, BEIJING, GUNAGHZOU],
    GUNAGHZOU: [WUHAN, BANGKOK],
    SHENZHEN: [WUHAN, BANGKOK],
    BANGKOK: [SHENZHEN, GUNAGHZOU]
           }

In [4]:
import networkx as nx

In [5]:
graph = connection

In [6]:
g=nx.Graph(graph)

In [7]:
nx.draw(g)

D:\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [8]:
def NAV(st,des,map):
    Queue=[[st]]     # FIFO 
    
    while (Queue) :
        path=Queue.pop(0) # pathes poped element defined as a list  
        node=path[-1] # last element choice .list index by [] 
        successor=map[node] # subnode from node
        
        for e in successor:
            if(e==des):
                path.append(e)
                return path
            else:
                Queue.append(path+[e])


In [9]:
def draw_route(routes):
    print('✈->'.join(routes))

In [10]:
draw_route(NAV(CHANGCHUN, SHENZHEN, connection))

CHANGCHUN✈->BEIJING✈->SHENZHEN


# 2. 语句自动生成


增加“=”号运算符

In [11]:
decimal_grammar = """
expression => operator op operator equal operator  
operator => num
num => 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | num num
op => + | - | * | /
equal => =
"""

In [12]:
grammar = """
sentence => noun_phrase verb_phrase 
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的
"""

In [13]:
import random

In [14]:
def parse_grammar(grammar_str,sep='=>'):
    grammar = {} #定义字典
    for line in grammar_str.split('\n'):
        line = line.strip()  #移除空格隔开的字符
        if not line: continue
            
        target,rule = line.split(sep) 
        
        grammar[target.strip()]=[r.split() for r in rule.split('|')] 
    return grammar

In [15]:
def gene(grammar_parsed,target='sentence'):
    if target not in grammar_parsed: return target
    rules = random.choice(grammar_parsed[target])
    return '' .join(gene(grammar_parsed,target=r) for r in rules if r!='null')

In [16]:
g=parse_grammar(grammar)

In [17]:
print (gene(g))

这个蓝色的好看的小猫看着一个蓝色的好看的篮球


In [18]:
artim=parse_grammar(decimal_grammar,sep='=>')

In [19]:
print (gene(artim,target='expression'))

3+2=7


# 问题1：第一个人机对话程序

In [20]:
import random
import jieba
import re
from fnmatch import fnmatch

In [21]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?y', '如果你能?y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}


In [22]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:]) # all() 判定所有元素是否均为true  isalpha()判定是否为字母字符且不留空

In [23]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [24]:
def map_ruleskey(speech,patten):

    string=re.sub(r'\?\*.','*',patten)
    string=string.strip()
    if(string.startswith('*')):
        string=string.replace(' ','',1)
    if(string.endswith('*')):
        pos=string.rfind(' ')
        string=string[:pos]+string[pos:].replace(' ','',1)
    return fnmatch(speech,string)
    

In [25]:
def subsitite(rule, parsed_rules): 
    if not rule: return [] 
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules) #便历rule列表,如果字典有匹配项，则替换，没有则返回当前值

In [26]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [27]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    
    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)
        

In [28]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [29]:
def get_response(saying, rules=rule_responses):
    got_pattern=[]    
    for pat_key in rules.keys():
         if map_ruleskey(saying,pat_key):
            got_pattern=pat_match_with_seg(pat_key.split(),saying.split())
            break
#    print(got_pattern)
    rule_response=random.choice(rules[pat_key])
    
    return ' '.join(subsitite(rule_response.split(), pat_to_dict(got_pattern)))

In [30]:
get_response("hello")

'Please state your problem'

In [31]:
get_response("I feel nervous")

'Do you often feel nervous ?'

In [32]:
get_response("I want that shoes")

'Why do you want that shoes'

# 问题2：中文对话

In [33]:
def parse_pat(patten):

    seg_list = jieba.cut(patten, cut_all=False)
    
    string=list(seg_list)  #'*?x我?*z梦见?*y'

    strlist=[]
    s=''

    for e in string:
        if not '\u4e00'<=e<='\u9fa5' : #判断e 是否为中文
            s=s+e 
            continue;
        if s :
            strlist.append(s)
            s=''
        strlist.append(e) 
    if s :
        strlist.append(s)
        
    return  strlist

In [34]:
def segment_match_cn(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    
    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

In [35]:
fail = [True, None]

def pat_match_with_seg_cn(pattern, saying):
    if not pattern or not saying: return []  
    pat = pattern[0]    
    if pat.startswith('?*'):
        match, index = segment_match_cn(pattern, saying)
#        print (match,index)
        return [match] + pat_match_with_seg_cn(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg_cn(pattern[1:], saying[1:])
    else:
        return fail

In [36]:
def map_ruleskey_cn(speech,patten):

    string=re.sub(r'\?\*.','*',patten)
    string=string.strip()
 #   if(string.startswith('*')):
 #       string=string.replace(' ','',1)
 #   if(string.endswith('*')):
 #       pos=string.rfind(' ')
 #       string=string[:pos]+string[pos:].replace(' ','',1)
    return fnmatch(speech,string)
    

In [37]:
def get_response_cn(saying, rules=rule_responses):
    got_pattern=[]    
    instrunction=list(jieba.cut(saying))
    for pat_key in rules.keys():
         if map_ruleskey_cn(saying,pat_key):
            patten=list(jieba.cut(pat_key))
            got_pattern=pat_match_with_seg_cn(parse_pat(pat_key),instrunction)
            break
#    print (got_pattern)
    rule_response=random.choice(rules[pat_key])
    return ''.join(subsitite(parse_pat(rule_response), pat_to_dict(got_pattern)))
   
    

In [38]:
get_response_cn('我很难过')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.681 seconds.
Prefix dict has been built succesfully.


'我听到你这么说， 也很难过'

In [39]:
get_response_cn('你为什么不踢足球')

'你觉得我不会踢足球'

In [40]:
get_response_cn('AI你好')

'你好呀'

In [41]:
get_response_cn('昨晚我梦见大海')

'你以前梦见过大海吗'

# 问题3：更好玩的人机对话

中文对话基础上添加数字运算：
例如：“2只猫加3只猫等于多少”=》“答案是 5”

In [42]:
rule_mix = {
    '?*m ?*op ?*n 等于 ?*z': ['答案是 ?m ?n ?op'],
    '?*m ?*op ?*n 一共 ?*z': ['答案是 ?m ?n ?op']
}

In [43]:
opr={'加':'+','减':'-'}

In [44]:
def parse_pat(patten):

    seg_list = jieba.cut(patten, cut_all=False)
    
    string=list(seg_list)  

    strlist=[]
    s=''

    for e in string:
        if  not '\u4e00'<=e<='\u9fa5' and not e==' ' : #e 不是中文 且 e不为空格
            s=s+e 
            continue;
        if s :
            strlist.append(s)
            s=''
        strlist.append(e) 
    if s :
        strlist.append(s)
    for i in strlist :   #删除列表中所有空格' '
        if ' ' in strlist:
            strlist.remove(' ')
            
    return  strlist

In [45]:
def segment_match_mix(pattern, saying, mode):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if mode=='numeric':
            if token.isnumeric():
                return (seg_pat, saying[i]),  i
        elif mode== 'operation': 
            if token=='加' or token=='减':
                symbol=opr[token]
                return (seg_pat, symbol), i
        else :
            if rest[0] == token:  
                return (seg_pat, saying[:i]), i
   
    return (seg_pat, saying), len(saying)

In [46]:
fail = [True, None]

def pat_match_with_seg_num(pattern, saying):
    if not pattern or not saying: return []  
    pat = pattern[0]    
    if pat=='?*n'or pat=='?*m'  :
        match, index = segment_match_mix(pattern, saying,'numeric')
        return [match] + pat_match_with_seg_num(pattern[1:], saying[index:])
    if pat=='?*op':
        match, index = segment_match_mix(pattern, saying,'operation')
        return [match] + pat_match_with_seg_num(pattern[1:], saying[index:])
    if pat=='?*x' or pat=='?*y' or pat=='?*z'  :
        match, index = segment_match_mix(pattern, saying,'textual')
        return [match] + pat_match_with_seg_num(pattern[1:], saying[index:])
    if pat == saying[0]:
        return pat_match_with_seg_num(pattern[1:], saying[1:])
    else:
        return pat_match_with_seg_num(pattern, saying[1:])

In [47]:
def map_ruleskey_num(speech,patten):

    string=re.sub(r'\?\*[^o]','*',patten)
    string=re.sub(r'\?\*[o][p]','[\u52a0|\u51cf]',string)
    string=re.sub(' ','',string)
 
    return fnmatch(speech,string)

In [48]:
t=[]

def subsitite_mix(rule, parsed_rules): 
    if not rule: return []
 
    g=parsed_rules.get(rule[0],rule[0])
    
    if rule[0]=='?m' or rule[0]=='?n':
        t.append(int(g))
        return subsitite_mix(rule[1:], parsed_rules)
    if rule[0]=='?op':
        if g=='+': 
            g=str(t.pop()+t.pop())
        if g=='-':
            g=str(-t.pop()+t.pop())
    return [g] + subsitite_mix(rule[1:], parsed_rules) 

In [49]:
#subsitite_mix(parse_pat('答案是 ?m ?n ?op'),pat_to_dict([('?m', '5'), ('?op', '-'), ('?n', '3'), ('?z', ['几只'])]))

In [50]:
def get_response_mix(saying, rules=rule_mix):
    got_pattern=[]    
    instruction=list(jieba.cut(saying,cut_all=True))
#    print (instruction)
    for pat_key in rules.keys():
        if map_ruleskey_num(saying,pat_key):

            got_pattern=pat_match_with_seg_num(parse_pat(pat_key),instruction)
            break
#    print (got_pattern)

    return ''.join(subsitite_mix(parse_pat(rules[pat_key][0]), pat_to_dict(got_pattern)))
   

In [51]:
get_response_mix('5只猫减3只猫一共几只猫?')

'答案是2'

In [52]:
get_response_mix('16只猫加3只猫等于几只猫?')

'答案是19'

# 问题 4 ：关于以上程序的思考

1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 


上面的程序实现较为简单，能够实现基本的应答。但有几个主要问题：
1.字典匹配Key语句表达式是固定的，如果saying语句关键字或者顺序不对应，则无法应答及匹配。后续可采用形式化语言的方式进行匹配。
2.目前设计的对话缺少连续性，后续应通过能在对话基础上标注关键信息，在机器回答语句后面添加下一步应该获取应答的链接从而实现人机连续对话。

2.什么是数据驱动？数据驱动在这个程序里如何体现？

数据驱动是根据输入的数据对其进行组织形成信息，之后对相关的信息进行整合和提炼，在数据的基础上经过训练和拟合形成自动化的决策的过程。本程序中即利用字典匹配输入并提取关键字，再对关键字封装后进行输出。

3.数据驱动与 AI 的关系是什么？ 

In [ ]:
数据驱动是面向场景进行软件或系统设计的一种设计方法，而AI是数据加工提炼的工具。